In [1]:
import os
import random
import numpy as np
import sys
import torch
import pprint

root = '/'
pprint.pprint(sys.path)

sys.path.append(import_path)
pprint.pprint(sys.path)

['/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages']


In [2]:
import os
import sys
import pprint

import_path = root + 'pigvae_all'
sys.path.append(import_path)
pprint.pprint(sys.path)

import_path1 = root + 'pigvae_all/pigvae'
sys.path.append(import_path1)
pprint.pprint(sys.path)

['/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages',
 'pigvae_all']
['/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages',
 'pigvae_all',
 'ddpm-torch']


In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data.distributed import DistributedSampler
import random
#import pytorch_lightning as pl
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx
from networkx.algorithms.shortest_paths.dense import floyd_warshall_numpy

from networkx.generators.random_graphs import *
from networkx.generators.ego import ego_graph
from networkx.generators.geometric import random_geometric_graph

/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_spline_conv/_version_cpu.so: undefined symbol: _ZN

In [5]:
loaded_data = torch.load(root + 'dataset/train_dataset/zinc_gdata/tensor_data.pkl')

# 各テンソルにアクセス
node_features = loaded_data['node_features']
edge_features = loaded_data['edge_features']
mask = loaded_data['mask']
props = loaded_data['props']

In [6]:
num_node_f, num_edge_f = 43, 6

In [7]:
node_features.shape, edge_features.shape, mask.shape, props.shape

(torch.Size([75000, 26, 43]),
 torch.Size([75000, 26, 26, 13]),
 torch.Size([75000, 26]),
 torch.Size([75000]))

In [8]:
train_dataset = torch.utils.data.TensorDataset(node_features[0:50000], edge_features[0:50000], mask[0:50000], props[0:50000])
val_dataset = torch.utils.data.TensorDataset(node_features[50000:60000], edge_features[50000:60000], mask[50000:60000], props[50000:60000])
test_dataset = torch.utils.data.TensorDataset(node_features[60000:70000], edge_features[60000:70000], mask[60000:70000], props[60000:70000])

In [9]:
n_batch = 20

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=n_batch, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=n_batch, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=n_batch, shuffle=False)

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [11]:
hparams = {
    "vae":True,
    "kld_loss_scale":0.01,
    "perm_loss_scale":0.1,
    "property_loss_scale":0.5,
    "num_node_features":num_node_f,
    "num_edge_features":6+num_edge_f+1,
    "emb_dim": 90,
    'graph_encoder_hidden_dim': 256,
    'graph_encoder_k_dim': 64,
    'graph_encoder_v_dim': 64,
    'graph_encoder_num_heads': 16,
    'graph_encoder_ppf_hidden_dim': 512,
    'graph_encoder_num_layers': 16,
    'graph_decoder_hidden_dim': 256,
    'graph_decoder_k_dim': 64,
    'graph_decoder_v_dim': 64,
    'graph_decoder_num_heads': 16,
    'graph_decoder_ppf_hidden_dim': 512,
    'graph_decoder_num_layers': 16,
    "graph_decoder_pos_emb_dim": 64,
    'property_predictor_hidden_dim': 3,
    'num_properties': 1
}

model_save_dir = root + 'save_models/zinc/pig-beta-vae_models/'

if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

In [12]:
from pigvae.models import GraphAE

model = GraphAE(hparams).to(device)

In [13]:
from pigvae.synthetic_graphs.critic_dev import Critic

critic = Critic(hparams, device)

In [14]:
lr = 0.00005

optimizer = torch.optim.Adam(model.parameters(), lr = 0.00005, betas = (0.9, 0.98))
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer = optimizer, gamma = 0.9)

# Training

In [17]:
def val_preds(epoch):

    model.eval()

    sum_errors = 0

    for batch_idx, batch_data in enumerate(val_loader):

        with torch.no_grad():

            if torch.cuda.is_available():
                node_features, edge_features, mask, props = batch_data
                node_features, edge_features, mask, props = node_features.to(device), edge_features.to(device), mask.to(device), props.to(device)

            graph_pred, perm, mu, logvar = model(node_features, edge_features, mask, training=True, tau=1.0)
            node_logits, edge_logits, pred_props = graph_pred

            vae_loss = critic(node_features, edge_features, props, node_logits, edge_logits, pred_props, mask, perm, mu, logvar)

            batch_vae_loss = vae_loss['node_loss'] * 10 + vae_loss['edge_loss'] * 10 + vae_loss['kld_loss'] * 0.01 + vae_loss['perm_loss'] * 0.1

            if batch_idx % 10 == 0:
                print(vae_loss)
                print("Epoch (val) : ", epoch, "  batch (val) : ", batch_idx , "Loss sum : ", batch_vae_loss.item())

            sum_errors = sum_errors + vae_loss['node_loss'].data.item() * 10 + vae_loss['edge_loss'].data.item() * 10 + vae_loss['kld_loss'].data.item() * 0.01

            del batch_vae_loss
            torch.cuda.empty_cache()

    return sum_errors

In [18]:
import datetime

# テキストファイルのパスを指定
log_path = model_save_dir + 'log.txt'

f = open(log_path, 'w')
f.write('train -- logs')
f.close()

def logging_txt(epoch):
    dt_now = datetime.datetime.now()

    # ファイルを読み込んで、末尾の行を取得
    with open(log_path, 'r') as file:
        lines = file.readlines()
        last_line = lines[-1].strip()

    # 末尾の行に追記する文字列を指定
    append_string = "epoch " + str(epoch) + " : " + str(dt_now)

    # ファイルに追記
    with open(log_path, 'a') as file:
        file.write('\n' + append_string)

In [ ]:
train_loss_list = []
val_loss_list = []

num_epochs = 100
best_loss = 10000000

for epoch in range(0, num_epochs):
    model.train()

    sum_errors = 0

    for batch_idx, batch_data in enumerate(train_loader):

        node_features, edge_features, mask, props = batch_data
        node_features, edge_features, mask, props = node_features.to(device), edge_features.to(device), mask.to(device), props.to(device)

        optimizer.zero_grad()

        graph_pred, perm, mu, logvar = model(node_features, edge_features, mask, training=True, tau=1.0)
        node_logits, edge_logits, pred_props = graph_pred

        vae_loss = critic(node_features, edge_features, props, node_logits, edge_logits, pred_props, mask, perm, mu, logvar)

        batch_vae_loss = vae_loss['node_loss'] * 10 + vae_loss['edge_loss'] * 10 + vae_loss['kld_loss'] * 0.01 + vae_loss['perm_loss'] * 0.1

        if batch_idx % 100 == 0:
            print("Epoch : ", epoch, " Batch : ", batch_idx, " Loss : ", batch_vae_loss.data.item())
            print(vae_loss)

        sum_errors = sum_errors + vae_loss['node_loss'].data.item() * 10 + vae_loss['edge_loss'].data.item() * 10 + vae_loss['kld_loss'].data.item() * 0.01

        batch_vae_loss.backward()

        optimizer.step()

        del batch_vae_loss
        torch.cuda.empty_cache()

    lr_scheduler.step()

    train_loss_list.append(sum_errors / len(train_loader))
    val_loss = val_preds(epoch) / len(val_loader)
    val_loss_list.append(val_loss)

    np.save(model_save_dir + "train_loss_hist", np.array(train_loss_list))
    np.save(model_save_dir + "val_loss_hist", np.array(val_loss_list))

    logging_txt(epoch)

    if val_loss < best_loss:
        save_model_path = model_save_dir + "pigvae_best_model" + ".pt"
        torch.save(model, save_model_path)

        optimizer_path = model_save_dir + "opt_best_model" + ".pt"
        torch.save(optimizer.state_dict(), optimizer_path)

        best_loss = val_loss

Epoch :  0  Batch :  0  Loss :  14.480592727661133
{'edge_loss': tensor(0.6059, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.7764, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(10.2184, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(5.5577, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(490.2033, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  0  Batch :  10  Loss :  7.990706443786621
{'edge_loss': tensor(0.3513, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.3811, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(9.1321, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(5.7540, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(524.6596, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  0  Batch :  20  Loss :  7.438491344451904
{'edge_loss': tensor(0.3475, device='cuda:0', grad_fn=<DivBa

Epoch :  0  Batch :  190  Loss :  4.729964256286621
{'edge_loss': tensor(0.2531, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.1843, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(5.2559, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(3.0353, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(484.8361, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  0  Batch :  200  Loss :  4.558914661407471
{'edge_loss': tensor(0.2334, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.1873, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(3.4201, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(3.1743, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(510.1400, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  0  Batch :  210  Loss :  4.596376895904541
{'edge_loss': tensor(0.2480, device='cuda:0', grad_fn=<Div

Epoch :  0  Batch :  380  Loss :  3.587959051132202
{'edge_loss': tensor(0.2058, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.1156, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(5.9623, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(3.1404, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(507.9803, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  0  Batch :  390  Loss :  3.5168657302856445
{'edge_loss': tensor(0.1987, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.1163, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(4.8253, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(3.1876, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(503.5439, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  0  Batch :  400  Loss :  3.516817569732666
{'edge_loss': tensor(0.1981, device='cuda:0', grad_fn=<Di

Epoch :  0  Batch :  570  Loss :  3.2975234985351562
{'edge_loss': tensor(0.2030, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.0888, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(5.5265, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(3.2405, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(467.3899, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  0  Batch :  580  Loss :  3.2033767700195312
{'edge_loss': tensor(0.2040, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.0810, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(4.2406, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(3.1043, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(466.1311, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  0  Batch :  590  Loss :  3.0650877952575684
{'edge_loss': tensor(0.1809, device='cuda:0', grad_fn=<

Epoch :  0  Batch :  760  Loss :  2.807126522064209
{'edge_loss': tensor(0.1844, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.0593, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(3.9966, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(3.2976, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(517.2817, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  0  Batch :  770  Loss :  3.0799410343170166
{'edge_loss': tensor(0.1977, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.0735, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(4.8662, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(3.1912, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(470.9875, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  0  Batch :  780  Loss :  2.8450427055358887
{'edge_loss': tensor(0.1858, device='cuda:0', grad_fn=<D

...

Epoch :  39  Batch :  2260  Loss :  0.966190755367279
{'edge_loss': tensor(0.0680, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(-0.0006, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(2.6770, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(2.6607, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(474.2336, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  39  Batch :  2270  Loss :  1.0376625061035156
{'edge_loss': tensor(0.0646, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.0070, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(2.6590, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(2.9436, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(524.6417, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  39  Batch :  2280  Loss :  1.1344711780548096
{'edge_loss': tensor(0.0801, device='cuda:0', gra

Epoch :  39  Batch :  2450  Loss :  1.0941866636276245
{'edge_loss': tensor(0.0737, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.0057, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(2.8296, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(2.7236, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(479.7606, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  39  Batch :  2460  Loss :  1.089434027671814
{'edge_loss': tensor(0.0681, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.0104, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(2.6010, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(2.7835, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(488.2289, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  39  Batch :  2470  Loss :  1.0533500909805298
{'edge_loss': tensor(0.0689, device='cuda:0', grad

{'edge_loss': tensor(0.0708, device='cuda:0'), 'node_loss': tensor(0.0057, device='cuda:0'), 'kld_loss': tensor(2.7566, device='cuda:0'), 'perm_loss': tensor(2.7269, device='cuda:0'), 'property_loss': tensor(468.3546, device='cuda:0')}
Epoch (val) :  39   batch (val) :  200 Loss sum :  1.0646058320999146
{'edge_loss': tensor(0.0612, device='cuda:0'), 'node_loss': tensor(0.0067, device='cuda:0'), 'kld_loss': tensor(3.0871, device='cuda:0'), 'perm_loss': tensor(2.8011, device='cuda:0'), 'property_loss': tensor(499.5628, device='cuda:0')}
Epoch (val) :  39   batch (val) :  210 Loss sum :  0.9898141622543335
{'edge_loss': tensor(0.0558, device='cuda:0'), 'node_loss': tensor(0.0050, device='cuda:0'), 'kld_loss': tensor(2.3813, device='cuda:0'), 'perm_loss': tensor(2.7178, device='cuda:0'), 'property_loss': tensor(490.4807, device='cuda:0')}
Epoch (val) :  39   batch (val) :  220 Loss sum :  0.9035483598709106
{'edge_loss': tensor(0.0616, device='cuda:0'), 'node_loss': tensor(-0.0012, device

{'edge_loss': tensor(0.0663, device='cuda:0'), 'node_loss': tensor(-0.0022, device='cuda:0'), 'kld_loss': tensor(2.7978, device='cuda:0'), 'perm_loss': tensor(2.7083, device='cuda:0'), 'property_loss': tensor(465.8633, device='cuda:0')}
Epoch (val) :  39   batch (val) :  470 Loss sum :  0.9399127960205078
{'edge_loss': tensor(0.0707, device='cuda:0'), 'node_loss': tensor(-2.0932e-05, device='cuda:0'), 'kld_loss': tensor(2.7351, device='cuda:0'), 'perm_loss': tensor(2.7178, device='cuda:0'), 'property_loss': tensor(452.5929, device='cuda:0')}
Epoch (val) :  39   batch (val) :  480 Loss sum :  1.0060056447982788
{'edge_loss': tensor(0.0629, device='cuda:0'), 'node_loss': tensor(0.0042, device='cuda:0'), 'kld_loss': tensor(2.4244, device='cuda:0'), 'perm_loss': tensor(2.6762, device='cuda:0'), 'property_loss': tensor(479.3575, device='cuda:0')}
Epoch (val) :  39   batch (val) :  490 Loss sum :  0.9630305767059326
Epoch :  40  Batch :  0  Loss :  1.030240535736084
{'edge_loss': tensor(0.06

Epoch :  40  Batch :  170  Loss :  0.9577186107635498
{'edge_loss': tensor(0.0651, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.0022, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(2.5364, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(2.5965, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(469.5811, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  40  Batch :  180  Loss :  1.046044945716858
{'edge_loss': tensor(0.0682, device='cuda:0', grad_fn=<DivBackward0>), 'node_loss': tensor(0.0072, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'kld_loss': tensor(2.5500, device='cuda:0', grad_fn=<MeanBackward0>), 'perm_loss': tensor(2.6614, device='cuda:0', grad_fn=<AddBackward0>), 'property_loss': tensor(460.9430, device='cuda:0', grad_fn=<MseLossBackward0>)}
Epoch :  40  Batch :  190  Loss :  1.0955610275268555
{'edge_loss': tensor(0.0722, device='cuda:0', grad_fn